In [121]:
import os

import pandas as pd
import numpy as np
import plotly.express as px

from scipy.stats import pearsonr

# Table of Contents

1. [Data loading](#data-loading)
2. [General analysis](#general-analysis)
    - [System sizes](#system-sizes)
    - [System investments](#system-investments)
3. [Comparison](#comparison)
    - [Yearly savings](#yearly-savings)
    - [Energy costs](#energy-costs)
4. [Correlations](#correlations)
    - [On absolute savings](#on-absolute-savings)
    - [On relative savings](#on-relative-savings)

# Data loading

In [122]:
URI = os.getenv("DB_URI")

In [123]:
master = pd.read_sql("SELECT * FROM vea_industrial_load_profiles.master", URI)
master.set_index("id", inplace=True)
master.sort_index(inplace=True)

# in the authors database, the following columns are already present in master
# this will lead to problems later on, so we drop them here
# if you have created your own features by the "create_master_features.py", you
# do not need to worry about it
cols_to_drop = ["energy_costs_eur", "capacity_costs_eur", "total_costs_eur"]
for col_to_drop in cols_to_drop:
    try:
        master.drop(columns=col_to_drop, inplace=True)
    except KeyError as e:
        continue

master.head()

,sector_group_id,sector_group,capacity_price_over_2500h_eur_per_kw,energy_price_over_2500h_eur_per_kwh,capacity_price_under_2500h_eur_per_kw,energy_price_under_2500h_eur_per_kwh,zip_code,grid_level,number_of_peak_load,peak_load_kw,mean_load_kw,variance_kw,total_energy_kwh,full_load_hours_h,is_over_2500h,std_kw
id,,,,,,,,,,,,,,,,
0,B,Mining and quarrying,94.60,0.0060,14.67,0.0379,85,MS,1,2227.36,646.151302,423413.418622,5675793.04,2548.215394,True,650.702250
1,B,Mining and quarrying,96.11,0.0052,9.92,0.0397,91,MS,1,1096.00,208.694900,109792.561757,1833176.00,1672.605839,False,331.349607
2,B,Mining and quarrying,96.11,0.0052,9.92,0.0397,92,MS,1,816.00,129.444604,36662.220510,1137041.40,1393.433088,False,191.473812
3,B,Mining and quarrying,96.11,0.0052,9.92,0.0397,85,MS,1,701.60,114.201389,19954.089684,1003145.00,1429.796180,False,141.258946
4,C,Manufacturing industry,101.04,0.0078,12.60,0.0432,30,MS,2,3480.00,1845.483607,610080.465591,16210728.00,4658.255172,True,781.076479


In [124]:
baseline = pd.read_sql("SELECT * FROM vea_results.overview WHERE name LIKE '%%base%%'", URI)
baseline["id"] = baseline["name"].str.split("_").str[0].astype(int)
baseline.set_index("id", inplace=True)
baseline.sort_index(inplace=True)
baseline.head()

,name,energy_costs_eur,grid_energy_costs_eur,grid_capacity_costs_eur,grid_capacity_kw,storage_invest_eur,storage_annuity_eur,storage_capacity_kwh,inverter_invest_eur,inverter_annuity_eur,inverter_capacity_kw,solar_invest_eur,solar_annuity_eur,solar_capacity_kwp,total_yearly_costs_eur,total_annuity_eur,total_invest_eur
id,,,,,,,,,,,,,,,,,
0,0_baseline,1.021643e+06,271558.090967,41249.388603,2227.36,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.334450e+06,0.0,0.0
1,1_baseline,3.299717e+05,12033.847188,132976.873344,1096.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.749824e+05,0.0,0.0
2,2_baseline,2.046675e+05,7464.085529,99004.679424,816.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.111362e+05,0.0,0.0
3,3_baseline,1.805661e+05,6585.125290,85124.611622,701.60,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.722758e+05,0.0,0.0
4,4_baseline,2.917931e+06,884063.074775,55353.715200,3480.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.857348e+06,0.0,0.0


In [125]:
pv = pd.read_sql("SELECT * FROM vea_results.overview WHERE name LIKE '%%pv_only'", URI)
pv["id"] = pv["name"].str.split("_").str[0].astype(int)
pv.set_index("id", inplace=True)
pv.sort_index(inplace=True)
pv.head()

,name,energy_costs_eur,grid_energy_costs_eur,grid_capacity_costs_eur,grid_capacity_kw,storage_invest_eur,storage_annuity_eur,storage_capacity_kwh,inverter_invest_eur,inverter_annuity_eur,inverter_capacity_kw,solar_invest_eur,solar_annuity_eur,solar_capacity_kwp,total_yearly_costs_eur,total_annuity_eur,total_invest_eur
id,,,,,,,,,,,,,,,,,
0,0_pv_only,1.018851e+06,270816.128970,41249.388603,2227.360000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,1.330917e+06,0.000000,0.000000
1,1_pv_only,3.290701e+05,12000.967825,132976.873344,1096.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,4.740480e+05,0.000000,0.000000
2,2_pv_only,2.041083e+05,7443.691853,99004.679424,816.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,3.105566e+05,0.000000,0.000000
3,3_pv_only,1.800727e+05,6567.133144,85124.611622,701.600000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,2.717645e+05,0.000000,0.000000
4,4_pv_only,2.764451e+06,837562.247624,51356.823916,3228.721805,0.0,0.0,0.0,0.0,0.0,0.0,887819.548872,39641.073868,739.849624,3.693011e+06,39641.073868,887819.548872


# General analysis

In [126]:
total_profiles_analyzed = len(pv)
profiles_using_pv = pv[pv["solar_invest_eur"] > 0]
n_profiles_using_pv = len(profiles_using_pv)

print(f"{total_profiles_analyzed=}")
print(f"{n_profiles_using_pv=}")
print(f"Percentage of profiles using storage: {((n_profiles_using_pv / total_profiles_analyzed) * 100):.2f} %")

total_profiles_analyzed=1773
n_profiles_using_pv=1093
Percentage of profiles using storage: 61.65 %


## System sizes

In [127]:
print("")
solar_size = profiles_using_pv["solar_capacity_kwp"].copy()
solar_size.name = "PV system capacity"
print(solar_size.describe().drop("count").to_markdown())

system_size_fig_df = profiles_using_pv.copy()
system_size_fig_df = system_size_fig_df.rename(columns={
    "storage_capacity_kwh": "Storage",
    "inverter_capacity_kw": "Inverter",
    "solar_capacity_kwp": "PV system"})
system_size_fig = px.box(
    data_frame=system_size_fig_df,
    x="PV system",
    title="System sizes")
system_size_fig.update_layout(xaxis_title="Capacity in kWh (storage) / kW (inverter, PV)", yaxis_title="")
system_size_fig.update_xaxes(range=[0, 1000])
system_size_fig.show()


|      |   PV system capacity |
|:-----|---------------------:|
| mean |          245.117     |
| std  |          826.381     |
| min  |            0.0916031 |
| 25%  |           13.1805    |
| 50%  |           47.8056    |
| 75%  |          180.952     |
| max  |        20148.2       |


## System investments

In [128]:
solar_invest = profiles_using_pv["solar_invest_eur"]
solar_invest.name = "Absolute PV system investment"
print(solar_invest.describe().drop("count").to_markdown())

system_invest_fig_df = profiles_using_pv.copy()
system_invest_fig_df = system_invest_fig_df.rename(columns={
    "storage_invest_eur": "Storage",
    "inverter_invest_eur": "Inverter",
    "solar_invest_eur": "PV system"})
system_invest_fig = px.box(
    data_frame=system_invest_fig_df,
    x="PV system",
    title="System investments")
system_invest_fig.update_layout(xaxis_title="System investments in €", yaxis_title="")
system_invest_fig.update_xaxes(range=[0, 800e3])
system_invest_fig.show()

|      |   Absolute PV system investment |
|:-----|--------------------------------:|
| mean |                294141           |
| std  |                991657           |
| min  |                   109.924       |
| 25%  |                 15816.6         |
| 50%  |                 57366.8         |
| 75%  |                217143           |
| max  |                     2.41779e+07 |


# Comparison

In [129]:
abs_diff = baseline.drop(columns="name") - pv.drop(columns="name")

# drop those that could not be optimized
abs_diff.dropna(subset="total_yearly_costs_eur", inplace=True)

abs_diff.head()

,energy_costs_eur,grid_energy_costs_eur,grid_capacity_costs_eur,grid_capacity_kw,storage_invest_eur,storage_annuity_eur,storage_capacity_kwh,inverter_invest_eur,inverter_annuity_eur,inverter_capacity_kw,solar_invest_eur,solar_annuity_eur,solar_capacity_kwp,total_yearly_costs_eur,total_annuity_eur,total_invest_eur
id,,,,,,,,,,,,,,,,
0,2791.373626,741.961997,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,3533.335623,0.000000,0.000000
1,901.561967,32.879364,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,934.441331,0.000000,0.000000
2,559.200689,20.393676,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,579.594365,0.000000,0.000000
3,493.350000,17.992146,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,511.342146,0.000000,0.000000
4,153480.233255,46500.827151,3996.891284,251.278195,0.0,0.0,0.0,0.0,0.0,0.0,-887819.548872,-39641.073868,-739.849624,164336.877823,-39641.073868,-887819.548872


In [130]:
rel_diff = (baseline.drop(columns="name") - pv.drop(columns="name")) / baseline.drop(columns="name")

# drop those that could not be optimized
rel_diff.dropna(subset="total_yearly_costs_eur", inplace=True)

rel_diff

,energy_costs_eur,grid_energy_costs_eur,grid_capacity_costs_eur,grid_capacity_kw,storage_invest_eur,storage_annuity_eur,storage_capacity_kwh,inverter_invest_eur,inverter_annuity_eur,inverter_capacity_kw,solar_invest_eur,solar_annuity_eur,solar_capacity_kwp,total_yearly_costs_eur,total_annuity_eur,total_invest_eur
id,,,,,,,,,,,,,,,,
0,0.002732,0.002732,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.002648,NaN,NaN
1,0.002732,0.002732,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.001967,NaN,NaN
2,0.002732,0.002732,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.001863,NaN,NaN
3,0.002732,0.002732,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.001878,NaN,NaN
4,0.052599,0.052599,0.072206,0.072206,NaN,NaN,NaN,NaN,NaN,NaN,-inf,-inf,-inf,0.042604,-inf,-inf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2736,0.011529,0.011529,0.000582,0.000582,NaN,NaN,NaN,NaN,NaN,NaN,-inf,-inf,-inf,0.009477,-inf,-inf
2748,0.070263,0.070263,0.025722,0.025722,NaN,NaN,NaN,NaN,NaN,NaN,-inf,-inf,-inf,0.054460,-inf,-inf
2772,0.064696,0.064696,0.025558,0.025558,NaN,NaN,NaN,NaN,NaN,NaN,-inf,-inf,-inf,0.049909,-inf,-inf


## Yearly savings

In [131]:
abs_yearly_savings = abs_diff["total_yearly_costs_eur"]
abs_yearly_savings.name = "Total yearly savings in Eur"
print(abs_yearly_savings.describe().drop("count").to_markdown())

abs_tot_yearly_savings_fig_df = abs_diff.copy()
abs_tot_yearly_savings_fig_df = abs_tot_yearly_savings_fig_df.rename(columns={"total_yearly_costs_eur": "Savings"})
abs_tot_yearly_savings_fig = px.box(
    data_frame=abs_tot_yearly_savings_fig_df,
    x="Savings",
    title="Total yearly savings")
abs_tot_yearly_savings_fig.update_layout(xaxis_title="Total yearly savings in €", yaxis_title="")
abs_tot_yearly_savings_fig.update_xaxes(range=[0, 60e3])
abs_tot_yearly_savings_fig.show()

|      |   Total yearly savings in Eur |
|:-----|------------------------------:|
| mean |               31270.1         |
| std  |              129966           |
| min  |                   0           |
| 25%  |                 869.656       |
| 50%  |                3988.29        |
| 75%  |               16788.5         |
| max  |                   4.04296e+06 |


In [132]:
rel_yearly_savings = 100 * rel_diff["total_yearly_costs_eur"]
rel_yearly_savings.name = "Relative yearly savings in %"
print(rel_yearly_savings.describe().drop("count").to_markdown())

rel_yearly_savings_fig_df = rel_diff.copy() * 100
rel_yearly_savings_fig_df = rel_yearly_savings_fig_df.rename(columns={"total_yearly_costs_eur": "Savings"})
rel_yearly_savings_fig = px.box(
    data_frame=rel_yearly_savings_fig_df,
    x="Savings",
    title="Relative yearly savings")
rel_yearly_savings_fig.update_layout(xaxis_title="Relative yearly savings in %", yaxis_title="")
rel_yearly_savings_fig.update_xaxes(range=[0, 15])
rel_yearly_savings_fig.write_image("images/rel_yearly_savings_box.pdf")
rel_yearly_savings_fig.show()

|      |   Relative yearly savings in % |
|:-----|-------------------------------:|
| mean |                      2.80658   |
| std  |                      3.37228   |
| min  |                      0.0160091 |
| 25%  |                      0.2682    |
| 50%  |                      1.37999   |
| 75%  |                      4.21152   |
| max  |                     16.2929    |


## Energy costs

In [133]:
rel_yearly_energy_savings = 100 * rel_diff["energy_costs_eur"]
rel_yearly_energy_savings.name = "Relative yearly energy savings savings in %"
print(rel_yearly_energy_savings.describe().drop("count").to_markdown())

rel_yearly_energy_savings_fig_df = rel_diff.copy() * 100
rel_yearly_energy_savings_fig_df.rename(columns={"energy_costs_eur": "Savings"}, inplace=True)
rel_yearly_energy_savings_fig = px.box(
    data_frame=rel_yearly_energy_savings_fig_df,
    x="Savings",
    title="Relative yearly savings")
rel_yearly_energy_savings_fig.update_layout(xaxis_title="Relative yearly energy savings in %", yaxis_title="")
rel_yearly_energy_savings_fig.update_xaxes(range=[0, 20])
rel_yearly_energy_savings_fig.show()

|      |   Relative yearly energy savings savings in % |
|:-----|----------------------------------------------:|
| mean |                                      3.64824  |
| std  |                                      4.35313  |
| min  |                                      0.273224 |
| 25%  |                                      0.273224 |
| 50%  |                                      1.8562   |
| 75%  |                                      5.64572  |
| max  |                                     21.0772   |


# Correlations

## On absolute savings

In [134]:
# merge savings onto master (with features)
abs_diff_with_master = pd.merge(left=abs_diff, right=master, how="left", left_index=True, right_index=True)
abs_diff_with_master.head()

abs_diff_with_master["std_by_mean"] = abs_diff_with_master["std_kw"] / abs_diff_with_master["mean_load_kw"]
abs_diff_with_master["std_by_peak"] = abs_diff_with_master["std_kw"] / abs_diff_with_master["peak_load_kw"]
abs_diff_with_master["peak_by_mean"] = abs_diff_with_master["peak_load_kw"] / abs_diff_with_master["mean_load_kw"]

In [135]:
cols_to_drop = [
    "grid_level",
    "zip_code",
    "sector_group_id",
    "sector_group",
    "solar_invest_eur",
    "solar_annuity_eur",
    "solar_capacity_kwp"]
abs_correlations_df = abs_diff_with_master.drop(columns=cols_to_drop).corr()
px.imshow(abs_correlations_df, title="Correlation coefficients for total yearly savings")

In [136]:
abs_corr_fig_df = abs_correlations_df[["total_yearly_costs_eur"]].round(2)
abs_corr_fig_df.sort_values("total_yearly_costs_eur", inplace=True, ascending=False)
abs_corr_fig = px.bar(
    data_frame=abs_corr_fig_df,
    y="total_yearly_costs_eur",
    text_auto=True,
    title="Correlation between different load profile characteristics and total yearly savings")
abs_corr_fig.update_layout(yaxis_title="Correlation coefficient", xaxis_title="Variable")
abs_corr_fig.show()

In [137]:
df = pd.DataFrame()
i = 0
for var in abs_correlations_df.index:
    corr, p_value = pearsonr(y=abs_diff_with_master.dropna()["total_yearly_costs_eur"], x=abs_diff_with_master.dropna()[var])
    df.loc[i, "var"] = var
    df.loc[i, "corr"] = corr
    df.loc[i, "p_value"] = p_value
    i += 1

df.sort_values("corr", ascending=False, ignore_index=True)

/usr/lib/python3/dist-packages/scipy/stats/_stats_py.py:4781: ConstantInputWarning:

An input array is constant; the correlation coefficient is not defined.

/usr/lib/python3/dist-packages/scipy/stats/_stats_py.py:4781: ConstantInputWarning:

An input array is constant; the correlation coefficient is not defined.

/usr/lib/python3/dist-packages/scipy/stats/_stats_py.py:4781: ConstantInputWarning:

An input array is constant; the correlation coefficient is not defined.

/usr/lib/python3/dist-packages/scipy/stats/_stats_py.py:4781: ConstantInputWarning:

An input array is constant; the correlation coefficient is not defined.

/usr/lib/python3/dist-packages/scipy/stats/_stats_py.py:4781: ConstantInputWarning:

An input array is constant; the correlation coefficient is not defined.

/usr/lib/python3/dist-packages/scipy/stats/_stats_py.py:4781: ConstantInputWarning:

An input array is constant; the correlation coefficient is not defined.



,var,corr,p_value
0,total_yearly_costs_eur,1.000000,0.000000e+00
1,energy_costs_eur,0.998140,0.000000e+00
2,grid_energy_costs_eur,0.983971,0.000000e+00
3,grid_capacity_kw,0.756781,4.417235e-308
4,total_energy_kwh,0.656632,4.008584e-205
5,mean_load_kw,0.656632,4.008584e-205
6,peak_load_kw,0.572732,3.835981e-145
7,grid_capacity_costs_eur,0.488289,4.577228e-100
8,std_kw,0.487023,1.753795e-99
9,variance_kw,0.429455,2.226622e-75


## On relative savings

In [138]:
# merge savings onto master (with features)
rel_diff_with_master = pd.merge(left=rel_diff, right=master, how="left", left_index=True, right_index=True)
rel_diff_with_master.head()

rel_diff_with_master["std_by_mean"] = rel_diff_with_master["std_kw"] / rel_diff_with_master["mean_load_kw"]
rel_diff_with_master["std_by_peak"] = rel_diff_with_master["std_kw"] / rel_diff_with_master["peak_load_kw"]
rel_diff_with_master["peak_by_mean"] = rel_diff_with_master["peak_load_kw"] / rel_diff_with_master["mean_load_kw"]

In [139]:
cols_to_drop = [
    "grid_level",
    "zip_code",
    "sector_group_id",
    "sector_group",
    "solar_invest_eur",
    "solar_annuity_eur",
    "solar_capacity_kwp"]
rel_correlations_df = rel_diff_with_master.drop(columns=cols_to_drop).corr()
px.imshow(rel_correlations_df, title="Correlation coefficients for relative yearly savings")

In [140]:
rel_corr_fig_df = rel_correlations_df[["total_yearly_costs_eur"]].round(2)
rel_corr_fig_df.sort_values("total_yearly_costs_eur", inplace=True, ascending=False)
rel_corr_fig_df.dropna(inplace=True)
rel_corr_fig = px.bar(
    data_frame=rel_corr_fig_df,
    y="total_yearly_costs_eur",
    text_auto=True,
    title="Correlation between different load profile characteristics and relative yearly savings")
rel_corr_fig.update_layout(yaxis_title="Correlation coefficient", xaxis_title="Variable")
rel_corr_fig.show()

In [141]:
df = pd.DataFrame()
i = 0
for var in rel_correlations_df.index:
    if np.inf in rel_diff_with_master[var]:
        continue
    elif -np.inf in rel_diff_with_master[var]:
        continue
    elif np.nan in rel_diff_with_master[var]:
        continue
    elif rel_diff_with_master[var].isin([np.nan, np.inf, -np.inf]).any():
        continue
    if "storage" in var or "inverter" in var:
        continue

    corr, p_value = pearsonr(y=rel_diff_with_master["total_yearly_costs_eur"], x=rel_diff_with_master[var])
    df.loc[i, "var"] = var
    df.loc[i, "corr"] = corr
    df.loc[i, "p_value"] = p_value
    i += 1

df.sort_values("corr", ascending=False, ignore_index=True)

,var,corr,p_value
0,total_yearly_costs_eur,1.000000,0.000000e+00
1,energy_costs_eur,0.997125,0.000000e+00
2,grid_energy_costs_eur,0.997125,0.000000e+00
3,grid_capacity_costs_eur,0.678895,9.421956e-240
4,grid_capacity_kw,0.678895,9.421956e-240
5,full_load_hours_h,0.277326,1.183123e-32
6,is_over_2500h,0.263583,1.505706e-29
7,energy_price_over_2500h_eur_per_kwh,0.067850,4.271117e-03
8,total_energy_kwh,0.033340,1.606563e-01
9,mean_load_kw,0.033340,1.606563e-01


In [142]:
fig = px.histogram(rel_diff_with_master.copy() * 100, "total_yearly_costs_eur", color="is_over_2500h", title="Distributions of relative savings")
fig.update_xaxes(title="Savings in %")
fig.show()